# Boilerplate

In [1]:
import sys
sys.path.insert(0, 'venv/lib/python3.10/site-packages')
import asyncio
import time

from nwv2_python_wrapper import *
import nwv2_python_wrapper

# Run

List all Python wrappers

In [2]:
[f for f in dir(nwv2_python_wrapper) if not f.startswith('__') and f != 'nwv2_python_wrapper']

['BroadcastChatMessageW',
 'EndpointW',
 'FilterCmdW',
 'FilterInterface',
 'FilterModeW',
 'FilterNoticeW',
 'FilterRspW',
 'GameOptionsW',
 'GameOutcomeW',
 'GameUpdateW',
 'GenPartInfoW',
 'GenStateDiffPartW',
 'GenStateDiffW',
 'NetRegionW',
 'PacketSettingsW',
 'PacketW',
 'PlayerInfoW',
 'ProcessUniqueIdW',
 'RequestActionW',
 'ResponseCodeW',
 'RoomListW',
 'TransportCmdW',
 'TransportInterface',
 'TransportNoticeW',
 'TransportRspW',
 'UniUpdateW',
 'new_transport_interface']

## Transport layer setup

Create a Client and Server Transport layer and run each in the background.

In [3]:
SERVER_PORT = 62012

In [4]:
client_tiface_inner = await new_transport_interface("0.0.0.0", 0)
client_tiface_inner.run()

server_tiface_inner = await new_transport_interface("0.0.0.0", SERVER_PORT)
server_tiface_inner.run()

<Future pending cb=[<builtins.PyDoneCallback object at 0x7ff8fe2a29f0>()]>

## Wrap the PYO3 Transport Interface in a Python class with the same methods

Note that there is no `run()`. Hence, the passed in `TransportInterface` instance must have already had its `run()` method called.

In [5]:
class LoggingTransportInterface:
    def __init__(self, inner, tag):
        self.inner = inner
        self.ctr = 0
        self.skipped_count = None
        self.tag = tag
        
    async def command_response(self, transport_cmd):
        print(f"LTI({self.tag}){self.ctr}{self.skipped()}: received transport_cmd {transport_cmd}")
        self.ctr += 1
        transport_rsp = await self.inner.command_response(transport_cmd)
        print(f"LTI({self.tag}){self.ctr}: got transport_rsp back from Transport layer: {transport_rsp}")
        self.ctr += 1
        return transport_rsp
    
    def get_notifications(self):
        transport_notif_list = self.inner.get_notifications()
        if self.skipped_count is None:
            print(f"LTI({self.tag}){self.ctr}: got transport notifications: {transport_notif_list}")
            self.skipped_count = 0
            if len(transport_notif_list) > 0:
                self.reset()
        else:
            self.skipped_count += 1
        self.ctr += 1
        return transport_notif_list
    
    def skipped(self):
        if self.skipped_count is None:
            return ""
        count = self.skipped_count
        self.skipped_count = None
        return f" skipped {count}"
    
    def reset(self):
        self.skipped_count = None
        
client_tiface = LoggingTransportInterface(client_tiface_inner, "client")
client_tiface

server_tiface = LoggingTransportInterface(client_tiface_inner, "server")
server_tiface

# Filter layer setup

In [6]:
client_fiface = FilterInterface(client_tiface, FilterModeW("client"))
server_fiface = FilterInterface(server_tiface, FilterModeW("server"))

(client_fiface, server_fiface)

(<FilterInterface at 0x7ff8fe2bbcd0>, <FilterInterface at 0x7ff8fe2b9470>)

Find methods to run

In [7]:
[m for m in dir(client_fiface) if not m.startswith('__')]

['command_response', 'get_notifications', 'notif_poll_ms', 'run']

Run Client and Server Filter!

In [8]:
client_run_fut = client_fiface.run()
server_run_fut = server_fiface.run()

# After waiting a bit, the above future should not have completed
time.sleep(0.1)
assert not client_run_fut.done() and not server_run_fut.done()

LTI(client)0: got transport notifications: []
LTI(server)0: got transport notifications: []


## Filter layer operation

### As a client, connect to server

In [9]:
import random
id = random.randint(1,10000)
connect_action = RequestActionW("connect", name="FilterNotebookClient{}".format(id), client_version="0.0.666")
connect_action

Connect { name: "FilterNotebookClient3169", client_version: "0.0.666" }

In [10]:
jupyter_endpoint = EndpointW(f"127.0.0.1:{SERVER_PORT}")
filter_cmd = FilterCmdW("sendrequestaction", endpoint=jupyter_endpoint, action=connect_action)
filter_cmd

SendRequestAction { endpoint: Endpoint(127.0.0.1:62012), action: Connect { name: "FilterNotebookClient3169", client_version: "0.0.666" } }

In [11]:
await client_fiface.command_response(filter_cmd)

LTI(client)6 skipped 5: received transport_cmd SendPackets { endpoint: Endpoint(127.0.0.1:62012), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 0 }, retry_interval: 250ms }], packets: [Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient3169", client_version: "0.0.666" } }] }
LTI(client)7: got transport_rsp back from Transport layer: TransportRsp::EndpointError { error: Endpoint(127.0.0.1:62012) not found in transmit queue: Cannot update last_send }
LTI(client)8: got transport notifications: []


In [ ]:
client_notifications = client_fiface.get_notifications()
assert len(client_notifications) > 0
new_response_code = client_notifications[0]
new_response_code

In [ ]:
cookie = new_response_code.response_code.cookie
cookie

In [ ]:
server_notifications = server_fiface.get_notifications()
assert len(server_notifications) > 0
server_notifications[0]

### Convenience function

In [ ]:
async def request_action_to_filter(req_action, endpoint=chococat_endpoint):
    filter_cmd = FilterCmdW("sendrequestaction", endpoint=endpoint, action=req_action)
    return await client_fiface.command_response(filter_cmd)

### Join a room

In [ ]:
await request_action_to_filter(RequestActionW("joinroom", room_name="general"))

Due to a bug in the Netwayste v1 server, there is a `BadRequest { error_msg: "cannot join game because in-game" }` filter notice following the `JoinedRoom` filter notice, below. Apparently, it's caused by a packet retry of the `JoinRoom` request action by the Transport layer, which is erroneously being treated as a second request action by the NWv1 server. It's erroneous because the request sequence is the same, so it should have been dropped by the server.

In [ ]:
client_fiface.get_notifications()